# Manipulation des données de nuscenes avec panda

In [1]:
%matplotlib inline

from nuscenes.nuscenes import NuScenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# data/sets/nuscenes
# D:\Utilisateurs\Alexandre\Repertoire_D\nuscenes\v1 0-trainval01

nusc = NuScenes(version='v1.0-mini', dataroot='data/sets/nuscenes')
nusc_can = NuScenesCanBus(dataroot='data/sets/nuscenes')

#nusc2 = NuScenes(version='v1.0-trainval', dataroot='D:/Utilisateurs/Alexandre/Repertoire_D/nuscenes/v1.0-trainval01')


Bad key "text.kerning_factor" on line 4 in
C:\Users\Alexandre\anaconda3\envs\nu\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.7 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
nusc.list_scenes()

scene-0061, Parked truck, construction, intersectio... [18-07-24 03:28:47]   19s, singapore-onenorth, #anns:4622
scene-0103, Many peds right, wait for turning car, ... [18-08-01 19:26:43]   19s, boston-seaport, #anns:2046
scene-0655, Parking lot, parked cars, jaywalker, be... [18-08-27 15:51:32]   20s, boston-seaport, #anns:2332
scene-0553, Wait at intersection, bicycle, large tr... [18-08-28 20:48:16]   20s, boston-seaport, #anns:1950
scene-0757, Arrive at busy intersection, bus, wait ... [18-08-30 19:25:08]   20s, boston-seaport, #anns:592
scene-0796, Scooter, peds on sidewalk, bus, cars, t... [18-10-02 02:52:24]   20s, singapore-queensto, #anns:708
scene-0916, Parking lot, bicycle rack, parked bicyc... [18-10-08 07:37:13]   20s, singapore-queensto, #anns:2387
scene-1077, Night, big street, bus stop, high speed... [18-11-21 11:39:27]   20s, singapore-hollandv, #anns:890
scene-1094, Night, after rain, many peds, PMD, ped ... [18-11-21 11:47:27]   19s, singapore-hollandv, #anns:1762
sc

In [4]:
scene_name = 'scene-0061'
dic_scene = nusc_can.get_messages(scene_name,'vehicle_monitor')
#print(test)
features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
df_scene = pd.DataFrame.from_dict(dic_scene)[features]

print(df_scene)

    vehicle_speed  steering  throttle  left_signal  right_signal
0           31.44       3.0         0            0             0
1           30.73       3.5         0            1             0
2           29.45       3.6         0            1             0
3           28.09       3.6         0            0             0
4           27.07       3.5         0            0             0
5           26.25       3.6         0            1             0
6           25.35       4.0         0            0             0
7           24.51       4.7         0            0             0
8           23.58       6.5         0            1             0
9           22.69      12.6         0            1             0
10          21.89      23.8         0            0             0
11          21.20      34.4         0            0             0
12          20.25      45.8         0            1             0
13          19.34      69.8         0            1             0
14          18.33      87

In [5]:
scene_name = 'scene-1100'
my_scene_token = nusc.field2token('scene', 'name', scene_name)[0]
nusc.render_scene_channel(my_scene_token, 'CAM_FRONT')

## Premier test: essayer d'apprendre quand mettre le clignotant 
Note: juste un essai pour prendre en main le dataset, avec seulement la vitesse et l'inclinaison du volant je ne pense pas qu'il soit possible de prévoir quand mettre un clignotant, il manque certaines informations (trajectoire notamment)

### 1/ Prétraitement des données

In [7]:
all_scene = [ s["name"] for s in nusc.scene]
print(all_scene)

['scene-0061', 'scene-0103', 'scene-0553', 'scene-0655', 'scene-0757', 'scene-0796', 'scene-0916', 'scene-1077', 'scene-1094', 'scene-1100']


In [8]:
# Mets des 1 tout le temps pour le clignotant au lieu d'une alternance par défaut
def fill_signal(df,signal):
    i = 0
    index = df.columns.get_loc(signal)
    while df[signal][i] != 1:
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        df.iat[i,index] = 1
        i += 1
    return df

In [9]:
#Ajoute une nouvelle colonne où les valeurs sont: 0(rien), 1(clignotant gauche), 2(clignotant droit)
#Plus pratique "fill_signal" car on aura une seul colonne Y pour l'apprentissage
def add_signal_column(df,signal):
    i = 0
    tab = []
    while df[signal][i] != 1:
        tab += [0]
        i += 1
    while i  < len(df) and sum(df[signal][i:i+4]) >= 1:
        if signal == "right_signal":
            tab += [2]
        else:
            tab += [1]
        i += 1
    while i < len(df):
        tab += [0]
        i += 1
    df["signal"] = tab
    return df

In [10]:
blackint = nusc_can.can_blacklist
blacklist = [ "scene-0"+ str(i) for i in blackint]
print(blacklist)
print( "%s" in all_scene)

['scene-0161', 'scene-0162', 'scene-0163', 'scene-0164', 'scene-0165', 'scene-0166', 'scene-0167', 'scene-0168', 'scene-0170', 'scene-0171', 'scene-0172', 'scene-0173', 'scene-0174', 'scene-0175', 'scene-0176', 'scene-0309', 'scene-0310', 'scene-0311', 'scene-0312', 'scene-0313', 'scene-0314']
False


In [11]:
tab = []
for s in all_scene[:300]:
    #print(s)
    if s not in blacklist:
        dic_scene = nusc_can.get_messages(s,'vehicle_monitor')
        features = ["vehicle_speed","steering","throttle","left_signal","right_signal"]
        df_scene = pd.DataFrame.from_dict(dic_scene)[features]
        if df_scene["left_signal"].any():
            #df_scene = fill_signal(df_scene,"left_signal")
            new_df = add_signal_column(df_scene,"left_signal")
            #print(new_df)
        if df_scene["right_signal"].any():
            #df_scene = fill_signal(df_scene,"right_signal")
            new_df = add_signal_column(df_scene,"right_signal")
            #print(new_df)
        tab += [new_df]
    
df_total = pd.concat(tab)
print(df_total)

    vehicle_speed  steering  throttle  left_signal  right_signal  signal
0           31.44       3.0         0            0             0       0
1           30.73       3.5         0            1             0       1
2           29.45       3.6         0            1             0       1
3           28.09       3.6         0            0             0       1
4           27.07       3.5         0            0             0       1
..            ...       ...       ...          ...           ...     ...
35           0.00    -293.9         0            0             0       2
36           0.00    -293.9         0            0             0       2
37           0.00    -293.9         0            0             1       2
38           0.00    -293.9         0            0             1       2
39           0.00    -293.9         0            0             0       0

[390 rows x 6 columns]


### 2/ Apprentissage

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import svm, neighbors
import random

In [13]:
features = ["vehicle_speed","steering","throttle"]
X = df_total[features]
y = df_total["signal"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)

#### 2/1/ K plus proches voisins

In [14]:
model = neighbors.KNeighborsClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)


0.9615384615384616

0.80 avec les 1000 scènes (can bus), c'est pas mal pour un modèle basique, on peut améliorer ça

## Deuxième test: anticiper la vitesse avec un véhicule en face

Principe de base (idée): 
    - le système de pilotage envoie les postions gps à suivre, la vitesse et l'angle recommandés à l'algorithme de gestion
    - l'algorithme de gestion en fonction de la situation va envoyer sur un modèle pour donner une nouvelle valeur de la vitesse et de l'angle par rapport à l'environnement
    - plusieurs modèles, mais en premier un basique : s'il y a personne en face on change rien, sinon calcul de la nouvelle vitesse par rapport au véhicule devant.

Pour cela, il nous faut plusieurs données pour ce modèle:
    - position, vitesse, orientation du véhicule égo (nous) et du véhicule en face
    - à compléter?



### 1/ Recherche de ces données: